In [207]:
from sympy import *

N = MatrixSymbol('N', 2, 1)
m = MatrixSymbol('m', 2, 1)
Ir = MatrixSymbol('Ir', 2, 1)
Ic = MatrixSymbol('Ic', 2, 1)
Lc = MatrixSymbol('Lc', 2, 1)
L = MatrixSymbol('L', 2, 1)
q = MatrixSymbol('q', 2, 1)
qd = MatrixSymbol('\dot{q}', 2, 1)
qdd = MatrixSymbol('\ddot{q}', 2, 1)
tau = MatrixSymbol('tau', 2, 1)

# Unit vector (global coordinate system)
xhat = Matrix([1, 0])
yhat = Matrix([0, 1])

In [208]:
# absolute joint angles
a0 = q[0]
a1 = q[0] + q[1]
a = Matrix([a0, a1])

# unit vector of limb directions
e0  = cos(a0)*xhat + sin(a0)*yhat;
e1  = cos(a1)*xhat + sin(a1)*yhat;

# center of mass positions of link
G0 = Lc[0]*e0;
G1 = L[0]*e0 + Lc[1]*e1;

display(G0, G1, a)

Matrix([
[cos(q[0, 0])*Lc[0, 0]],
[sin(q[0, 0])*Lc[0, 0]]])

Matrix([
[cos(q[0, 0] + q[1, 0])*Lc[1, 0] + cos(q[0, 0])*L[0, 0]],
[sin(q[0, 0] + q[1, 0])*Lc[1, 0] + sin(q[0, 0])*L[0, 0]]])

Matrix([
[          q[0, 0]],
[q[0, 0] + q[1, 0]]])

In [209]:
def derivative(x):
    x = Matrix([x])
    return x.jacobian(Matrix([q,qd])) * Matrix([qd,qdd])

# time-derivatives of center of mass positions
dG0 = derivative(G0)
dG1 = derivative(G1)

# time-derivatives of absolute joint angles
da = derivative(a)

display(dG0, dG1, da)

Matrix([
[-sin(q[0, 0])*Lc[0, 0]*\dot{q}[0, 0]],
[ cos(q[0, 0])*Lc[0, 0]*\dot{q}[0, 0]]])

Matrix([
[(-sin(q[0, 0] + q[1, 0])*Lc[1, 0] - sin(q[0, 0])*L[0, 0])*\dot{q}[0, 0] - sin(q[0, 0] + q[1, 0])*Lc[1, 0]*\dot{q}[1, 0]],
[ (cos(q[0, 0] + q[1, 0])*Lc[1, 0] + cos(q[0, 0])*L[0, 0])*\dot{q}[0, 0] + cos(q[0, 0] + q[1, 0])*Lc[1, 0]*\dot{q}[1, 0]]])

Matrix([
[                \dot{q}[0, 0]],
[\dot{q}[0, 0] + \dot{q}[1, 0]]])

In [210]:
# Kinetic energy of structural parts
T0  = 0.5 * (m[0]*dG0.dot(dG0) + m[1]*dG1.dot(dG1))      # rectilinear
T1  = 0.5 * (Ic[0]*da[0]*da[0] + Ic[1]*da[1]*da[1])      # rotational
display(T0, T1)

0.5*(sin(q[0, 0])**2*Lc[0, 0]**2*\dot{q}[0, 0]**2 + cos(q[0, 0])**2*Lc[0, 0]**2*\dot{q}[0, 0]**2)*m[0, 0] + 0.5*(((-sin(q[0, 0] + q[1, 0])*Lc[1, 0] - sin(q[0, 0])*L[0, 0])*\dot{q}[0, 0] - sin(q[0, 0] + q[1, 0])*Lc[1, 0]*\dot{q}[1, 0])**2 + ((cos(q[0, 0] + q[1, 0])*Lc[1, 0] + cos(q[0, 0])*L[0, 0])*\dot{q}[0, 0] + cos(q[0, 0] + q[1, 0])*Lc[1, 0]*\dot{q}[1, 0])**2)*m[1, 0]

0.5*(\dot{q}[0, 0] + \dot{q}[1, 0])**2*Ic[1, 0] + 0.5*Ic[0, 0]*\dot{q}[0, 0]**2

In [211]:
# Kinetic energy of actuators, rotors run x(gearRatio) times faster than joints
dr0 = N[0]*qd[0]
dr1 = N[1]*qd[1]
T_act = 0.5*(Ir[0]*dr0*dr0 + Ir[1]*dr1*dr1)
display(dr0, dr1, T_act)

N[0, 0]*\dot{q}[0, 0]

N[1, 0]*\dot{q}[1, 0]

0.5*Ir[0, 0]*N[0, 0]**2*\dot{q}[0, 0]**2 + 0.5*Ir[1, 0]*N[1, 0]**2*\dot{q}[1, 0]**2

In [212]:
# Define Lagrangian
T = T0 + T1 + T_act     # Kinetic Energy
V = 0                   # Potential Energy 
Lag = T-V               # Lagrangian
Lag = simplify(nsimplify(Lag))

# Define Generalized Work
dW = tau[0]*qd[0] + tau[1]*qd[1]
display(Lag, dW)

(((sin(q[0, 0] + q[1, 0])*Lc[1, 0] + sin(q[0, 0])*L[0, 0])*\dot{q}[0, 0] + sin(q[0, 0] + q[1, 0])*Lc[1, 0]*\dot{q}[1, 0])**2 + ((cos(q[0, 0] + q[1, 0])*Lc[1, 0] + cos(q[0, 0])*L[0, 0])*\dot{q}[0, 0] + cos(q[0, 0] + q[1, 0])*Lc[1, 0]*\dot{q}[1, 0])**2)*m[1, 0]/2 + (\dot{q}[0, 0] + \dot{q}[1, 0])**2*Ic[1, 0]/2 + Ic[0, 0]*\dot{q}[0, 0]**2/2 + Ir[0, 0]*N[0, 0]**2*\dot{q}[0, 0]**2/2 + Ir[1, 0]*N[1, 0]**2*\dot{q}[1, 0]**2/2 + Lc[0, 0]**2*\dot{q}[0, 0]**2*m[0, 0]/2

\dot{q}[0, 0]*tau[0, 0] + \dot{q}[1, 0]*tau[1, 0]

In [213]:
# Euler-Lagrangian Equation of Motion (EoM)
EoM_LHS = zeros(2, 1)
for i in range(2):
    partial_dq_ = simplify(diff(Lag, qd[i]))
    partial_q_ = simplify(diff(Lag, q[i]))
    EoM_LHS[i] = derivative(partial_dq_)[0] - partial_q_

EoM_RHS = Matrix([dW]).jacobian(qd)
display(EoM_LHS, EoM_RHS)

Matrix([
[(-2*sin(q[1, 0])*L[0, 0]*Lc[1, 0]*\dot{q}[0, 0]*m[1, 0] - sin(q[1, 0])*L[0, 0]*Lc[1, 0]*\dot{q}[1, 0]*m[1, 0])*\dot{q}[1, 0] + (cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] + Ic[1, 0] + Lc[1, 0]**2*m[1, 0])*\ddot{q}[1, 0] + (2*cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] + Ic[0, 0] + Ic[1, 0] + Ir[0, 0]*N[0, 0]**2 + L[0, 0]**2*m[1, 0] + Lc[0, 0]**2*m[0, 0] + Lc[1, 0]**2*m[1, 0])*\ddot{q}[0, 0]],
[                                                                     (\dot{q}[0, 0] + \dot{q}[1, 0])*sin(q[1, 0])*L[0, 0]*Lc[1, 0]*\dot{q}[0, 0]*m[1, 0] + (cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] + Ic[1, 0] + Lc[1, 0]**2*m[1, 0])*\ddot{q}[0, 0] + (Ic[1, 0] + Ir[1, 0]*N[1, 0]**2 + Lc[1, 0]**2*m[1, 0])*\ddot{q}[1, 0] - sin(q[1, 0])*L[0, 0]*Lc[1, 0]*\dot{q}[0, 0]*\dot{q}[1, 0]*m[1, 0]]])

Matrix([[tau[0, 0], tau[1, 0]]])

In [214]:
M = EoM_LHS.jacobian(qdd)
c = EoM_LHS - M*qdd;
display(M, c)

Matrix([
[2*cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] + Ic[0, 0] + Ic[1, 0] + Ir[0, 0]*N[0, 0]**2 + L[0, 0]**2*m[1, 0] + Lc[0, 0]**2*m[0, 0] + Lc[1, 0]**2*m[1, 0], cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] + Ic[1, 0] + Lc[1, 0]**2*m[1, 0]],
[                                                                              cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] + Ic[1, 0] + Lc[1, 0]**2*m[1, 0],                   Ic[1, 0] + Ir[1, 0]*N[1, 0]**2 + Lc[1, 0]**2*m[1, 0]]])

Matrix([
[(-2*sin(q[1, 0])*L[0, 0]*Lc[1, 0]*\dot{q}[0, 0]*m[1, 0] - sin(q[1, 0])*L[0, 0]*Lc[1, 0]*\dot{q}[1, 0]*m[1, 0])*\dot{q}[1, 0] + (cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] + Ic[1, 0] + Lc[1, 0]**2*m[1, 0])*\ddot{q}[1, 0] + (2*cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] + Ic[0, 0] + Ic[1, 0] + Ir[0, 0]*N[0, 0]**2 + L[0, 0]**2*m[1, 0] + Lc[0, 0]**2*m[0, 0] + Lc[1, 0]**2*m[1, 0])*\ddot{q}[0, 0]],
[                                                                     (\dot{q}[0, 0] + \dot{q}[1, 0])*sin(q[1, 0])*L[0, 0]*Lc[1, 0]*\dot{q}[0, 0]*m[1, 0] + (cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] + Ic[1, 0] + Lc[1, 0]**2*m[1, 0])*\ddot{q}[0, 0] + (Ic[1, 0] + Ir[1, 0]*N[1, 0]**2 + Lc[1, 0]**2*m[1, 0])*\ddot{q}[1, 0] - sin(q[1, 0])*L[0, 0]*Lc[1, 0]*\dot{q}[0, 0]*\dot{q}[1, 0]*m[1, 0]]]) + Matrix([
[-2*cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] - Ic[0, 0] - Ic[1, 0] - Ir[0, 0]*N[0, 0]**2 - L[0, 0]**2*m[1, 0] - Lc[0, 0]**2*m[0, 0] - Lc[1, 0]**2*m[1, 0], -cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] - Ic[1, 0] - Lc[

In [215]:
nsimplify(M)

Matrix([
[2*cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] + Ic[0, 0] + Ic[1, 0] + Ir[0, 0]*N[0, 0]**2 + L[0, 0]**2*m[1, 0] + Lc[0, 0]**2*m[0, 0] + Lc[1, 0]**2*m[1, 0], cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] + Ic[1, 0] + Lc[1, 0]**2*m[1, 0]],
[                                                                              cos(q[1, 0])*L[0, 0]*Lc[1, 0]*m[1, 0] + Ic[1, 0] + Lc[1, 0]**2*m[1, 0],                   Ic[1, 0] + Ir[1, 0]*N[1, 0]**2 + Lc[1, 0]**2*m[1, 0]]])